In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # For the BERT tokenizer

# Load the BERT preprocessing and encoder layers from TensorFlow Hub
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

def build_model():
    # Input layer for text
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)

    # Preprocess the input text using the BERT preprocessing layer
    preprocessed_text = bert_preprocess(text_input)

    # Pass the preprocessed text to the BERT encoder
    encoder_outputs = bert_encoder(preprocessed_text)

    # Use the pooled_output (represents the [CLS] token's representation) for classification
    pooled_output = encoder_outputs['pooled_output']

    # Add a Dense layer for binary classification (positive/negative sentiment)
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(pooled_output)

    # Create the Keras model
    model = tf.keras.Model(inputs=[text_input], outputs=[output])

    return model

# Build and compile the model
model = build_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Check the model structure
model.summary()


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
model.save("../saved-model/sent-model1.h5") # saving the model

In [ ]:
# Ensure the model is in evaluation mode
model.trainable = False

In [ ]:
# Example texts for prediction
texts = ["I recently stayed at a hotel that was highly disappointing. The room was dirty, and the staff were unhelpful and rude. Despite requesting multiple times, the issues were never addressed. The amenities were outdated, and the overall experience was far below what I expected. I would not recommend this place to anyone and will avoid it in the future"]
# Create a TensorFlow dataset from the list of texts
dataset = tf.data.Dataset.from_tensor_slices(texts)
dataset = dataset.batch(1)  # Adjust the batch size as needed

# Perform prediction using the dataset
predictions = model.predict(dataset, steps=len(texts))  # Specify steps based on the number of texts

# Print the predictions
print(predictions)

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[0.42563742]]
